<a href="https://colab.research.google.com/github/Nazarique/meta-heur-sticas-bioinspiradas/blob/main/Lista_2/Lista_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://pypi.org/project/swarmlib/#ant-colony-optimization

https://pypi.org/project/ACO-Pants/

https://github.com/rochakgupta/aco-tsp

In [ ]:
pip install ACO-Pants

  Created wheel for ACO-Pants: filename=ACO_Pants-0.5.2-cp36-none-any.whl size=18854 sha256=4bdcf20b92e2ec016dd04ba99529f5277c562cc62f31fe65212cea0c4436b2e5
  Stored in directory: /root/.cache/pip/wheels/96/54/43/24aef81d408a74657e0cc72e811dd69b4a8f1d20c3ae758129
Successfully built ACO-Pants


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pants
import math
import random
import pandas as pd

In [ ]:
data = pd.read_csv("/content/drive/My Drive/8° Período/Meta/lista/lista 2/Omã.csv") 

In [ ]:
data.head(4)

,n,x,y
0,1,16700.0000,53233.3333
1,2,16700.0000,53250.0000
2,3,16716.6667,53300.0000
3,4,16733.3333,53333.3333


In [ ]:
data.iloc[-1]

n     1979.0000
x    26371.3889
y    56358.6111
Name: 1978, dtype: float64

In [ ]:
nodes = list(zip(data['x'], data['y']))

In [ ]:
# nodes = []
# for _ in range(20):
#   x = random.uniform(-10, 10)
#   y = random.uniform(-10, 10)
#   nodes.append((x, y))

In [ ]:
print(type(nodes))

<class 'list'>


In [ ]:
def euclidean(a, b):
    return math.sqrt(pow(a[1] - b[1], 2) + pow(a[0] - b[0], 2))

In [ ]:
world = pants.World(nodes, euclidean)

NameError: ignored

In [ ]:
solver = pants.Solver()

In [ ]:
solution = solver.solve(world)
# or
solutions = solver.solutions(world)

In [ ]:
print(solution.distance)
print(solution.tour)    # Nodes visited in order
print(solution.path)    # Edges taken in order
# or
best = float("inf")
for solution in solutions:
  assert solution.distance < best
  best = solution.distance

197799.6721078564
[(23076.1111, 58570.2778), (23084.4444, 58567.7778), (23098.3333, 58573.6111), (23103.3333, 58567.7778), (23107.2222, 58561.1111), (23115.5556, 58513.0556), (23112.5, 58516.1111), (23104.1667, 58580.5556), (23070.2778, 58614.4444), (23071.6667, 58610.5556), (23052.5, 58615.8333), (23048.0556, 58623.0556), (23050.5556, 58623.8889), (23066.6667, 58616.6667), (23046.1111, 58608.6111), (23045.8333, 58654.7222), (23046.9444, 58661.1111), (23047.5, 58688.0556), (23040.8333, 58701.6667), (23081.9444, 58725.2778), (23024.4444, 58743.0556), (23019.4444, 58740.0), (22880.5556, 58775.0), (22911.6667, 58880.8333), (22906.6667, 58883.0556), (22950.0, 58816.6667), (22966.6667, 58800.0), (22983.3333, 58783.3333), (22966.6667, 58550.0), (22978.6111, 58521.6667), (22987.5, 58510.8333), (22981.1111, 58535.5556), (22983.3333, 58550.0), (22966.6667, 58566.6667), (22983.3333, 58566.6667), (23123.6111, 58463.3333), (23114.4444, 58476.3889), (23101.6667, 58454.4444), (23118.0556, 58541.6667

# Aco

In [ ]:
import math
import random
import time as tm
from tqdm import tqdm
from matplotlib import pyplot as plt


class SolveTSPUsingACO:
    class Edge:
        def __init__(self, a, b, weight, initial_pheromone):
            self.a = a
            self.b = b
            if weight == 0:
                weight = 1e-10
            self.weight = weight
            self.pheromone = initial_pheromone

    class Ant:
        def __init__(self, alpha, beta, num_nodes, edges):
            self.alpha = alpha
            self.beta = beta
            self.num_nodes = num_nodes
            self.edges = edges
            self.tour = None
            self.distance = 0.0

        def _select_node(self):
            roulette_wheel = 0.0
            unvisited_nodes = [node for node in range(self.num_nodes) if node not in self.tour]
            heuristic_total = 0.0
            for unvisited_node in unvisited_nodes:
                heuristic_total += self.edges[self.tour[-1]][unvisited_node].weight
            for unvisited_node in unvisited_nodes:
                roulette_wheel += pow(self.edges[self.tour[-1]][unvisited_node].pheromone, self.alpha) * \
                                  pow((heuristic_total / self.edges[self.tour[-1]][unvisited_node].weight), self.beta)
            random_value = random.uniform(0.0, roulette_wheel)
            wheel_position = 0.0
            for unvisited_node in unvisited_nodes:
                wheel_position += pow(self.edges[self.tour[-1]][unvisited_node].pheromone, self.alpha) * \
                                  pow((heuristic_total / self.edges[self.tour[-1]][unvisited_node].weight), self.beta)
                if wheel_position >= random_value:
                    return unvisited_node

        def find_tour(self):
            self.tour = [random.randint(0, self.num_nodes - 1)]
            while len(self.tour) < self.num_nodes:
                self.tour.append(self._select_node())
            return self.tour

        def get_distance(self):
            self.distance = 0.0
            for i in range(self.num_nodes):
                self.distance += self.edges[self.tour[i]][self.tour[(i + 1) % self.num_nodes]].weight
            return self.distance

    def __init__(self, mode='ACS', colony_size=10, elitist_weight=1.0, min_scaling_factor=0.001, alpha=1.0, beta=3.0,
                 rho=0.1, pheromone_deposit_weight=1.0, initial_pheromone=1.0, steps=100, nodes=None, labels=None):
        self.mode = mode
        self.colony_size = colony_size
        self.elitist_weight = elitist_weight
        self.min_scaling_factor = min_scaling_factor
        self.rho = rho
        self.pheromone_deposit_weight = pheromone_deposit_weight
        self.steps = steps
        self.best_formigas = 0
        self.best_iter = 0
        self.num_nodes = len(nodes)
        self.nodes = nodes
        if labels is not None:
            self.labels = labels
        else:
            self.labels = range(1, self.num_nodes + 1)
        self.edges = [[None] * self.num_nodes for _ in range(self.num_nodes)]
        for i in range(self.num_nodes):
            for j in range(i + 1, self.num_nodes):
                self.edges[i][j] = self.edges[j][i] = self.Edge(i, j, math.sqrt(
                    pow(self.nodes[i][0] - self.nodes[j][0], 2.0) + pow(self.nodes[i][1] - self.nodes[j][1], 2.0)),
                                                                initial_pheromone)
        self.ants = [self.Ant(alpha, beta, self.num_nodes, self.edges) for _ in range(self.colony_size)]
        self.global_best_tour = None
        self.global_best_distance = float("inf")

    def _add_pheromone(self, tour, distance, weight=1.0):
        pheromone_to_add = self.pheromone_deposit_weight / distance
        for i in range(self.num_nodes):
            self.edges[tour[i]][tour[(i + 1) % self.num_nodes]].pheromone += weight * pheromone_to_add

    def _acs(self):
        for step in range(self.steps):
            for ant in self.ants:
                # Amout of pheromone increment is constant.
                self._add_pheromone(ant.find_tour(), ant.get_distance())
                if ant.distance < self.global_best_distance:
                    self.global_best_tour = ant.tour
                    self.global_best_distance = ant.distance
                    self.best_iter = step
                    self.best_formigas = self.ants
            for i in range(self.num_nodes):
                for j in range(i + 1, self.num_nodes):
                    self.edges[i][j].pheromone *= (1.0 - self.rho)

    def _elitist(self):
        for step in range(self.steps):
            for ant in self.ants:
                self._add_pheromone(ant.find_tour(), ant.get_distance())
                if ant.distance < self.global_best_distance:
                    self.global_best_tour = ant.tour
                    self.global_best_distance = ant.distance
                    self.best_iter = step
                    self.best_formigas = self.ants

            # Amout of pheromone depends upon the quality of the solution.
            self._add_pheromone(self.global_best_tour, self.global_best_distance, weight=self.elitist_weight)
            for i in range(self.num_nodes):
                for j in range(i + 1, self.num_nodes):
                    self.edges[i][j].pheromone *= (1.0 - self.rho)

    def _max_min(self):
        for step in range(self.steps):
            iteration_best_tour = None
            iteration_best_distance = float("inf")
            for ant in self.ants:
                ant.find_tour()
                if ant.get_distance() < iteration_best_distance:
                    iteration_best_tour = ant.tour
                    iteration_best_distance = ant.distance
            if float(step + 1) / float(self.steps) <= 0.75:
                # Initially higher weight gradually reduced will 75% completion
                self._add_pheromone(iteration_best_tour, iteration_best_distance)
                max_pheromone = self.pheromone_deposit_weight / iteration_best_distance
            else:
                # Last 25% weights based on quality of route after comparing with the global best tour.
                if iteration_best_distance < self.global_best_distance:
                    self.global_best_tour = iteration_best_tour
                    self.global_best_distance = iteration_best_distance
                    self.best_iter = step
                    self.best_formigas = self.ants

                self._add_pheromone(self.global_best_tour, self.global_best_distance)
                max_pheromone = self.pheromone_deposit_weight / self.global_best_distance
            min_pheromone = max_pheromone * self.min_scaling_factor

            # Setting edge weights to min and max phermones
            for i in range(self.num_nodes):
                for j in range(i + 1, self.num_nodes):
                    self.edges[i][j].pheromone *= (1.0 - self.rho)
                    if self.edges[i][j].pheromone > max_pheromone:
                        self.edges[i][j].pheromone = max_pheromone
                    elif self.edges[i][j].pheromone < min_pheromone:
                        self.edges[i][j].pheromone = min_pheromone

    def run(self):
        start = tm.time()
        # print('Started : {0}'.format(self.mode))
        if self.mode == 'ACS':
            self._acs()
        elif self.mode == 'Elitist':
            self._elitist()
        else:
          
            self._max_min()
        print('Ended : {0}'.format(self.mode))
        print('Sequence : <- {0} ->'.format(' - '.join(str(self.labels[i]) for i in self.global_best_tour)))
        print('Total distance travelled to complete the tour : {0}\n'.format(round(self.global_best_distance, 2)))
        runtime = tm.time() - start
        return runtime, self.global_best_distance, self.best_iter

    def plot(self, line_width=1, point_radius=math.sqrt(2.0), annotation_size=8, dpi=120, save=True, name=None):
        x = [self.nodes[i][0] for i in self.global_best_tour]
        x.append(x[0])
        y = [self.nodes[i][1] for i in self.global_best_tour]
        y.append(y[0])
        plt.plot(x, y, linewidth=line_width)
        plt.scatter(x, y, s=math.pi * (point_radius ** 2.0))
        plt.title(self.mode)
        for i in self.global_best_tour:
            plt.annotate(self.labels[i], self.nodes[i], size=annotation_size)
        if save:
            if name is None:
                name = '{0}_tour.png'.format(self.mode)
            plt.savefig(name, dpi=dpi)
        plt.show()
        plt.gcf().clear()


if __name__ == '__main__':
    _colony_size = 10
    _steps = 100
    
    f = open('/content/drive/My Drive/8° Período/Meta/lista/lista 2/resultados/Omã_Elitist_out.csv', 'w')

    f.write(','.join(['Elitist_Iteration', 'Elitist_time', 'Elitist_dist']))#, 'Elitist_Iteration', 'Elitist_time', \
        #'Elitist_dist', 'MaxMin_Iteration', 'MaxMin_time', 'MaxMin_dist'
    f.write('\n')
    # Analyse and compare the various modes

    _nodes = nodes
    #acs = SolveTSPUsingACO(mode='ACS', colony_size=_colony_size, steps=_steps, nodes=_nodes)
    elitist = SolveTSPUsingACO(mode='Elitist', colony_size=_colony_size, steps=_steps, nodes=_nodes)
    #max_min = SolveTSPUsingACO(mode='MaxMin', colony_size=_colony_size, steps=_steps, nodes=_nodes)

    time1, dist1, iter1 = elitist.run()
    route1 = elitist.global_best_tour
    formigueiro1 = elitist.best_formigas
    f.write(str(iter1) + "," + str(time1) + "," + str(dist1) + ",")

    # time2, dist2, iter2 = elitist.run()
    # route2 = elitist.global_best_tour
    # formigueiro2 = elitist.best_formigas
    # f.write(str(iter2) + ","+ str(time2) + "," + str(dist2) + ",")

    # time3, dist3, iter3 = max_min.run()
    # route3 = max_min.global_best_tour
    # formigueiro3 = max_min.best_formigas
    # f.write(str(iter3) + "," + str(time3) + "," + str(dist3) + "\n")/

    f.close()

Ended : Elitist
Sequence : <- 160 - 154 - 131 - 119 - 117 - 89 - 83 - 68 - 66 - 77 - 86 - 85 - 64 - 65 - 87 - 31 - 33 - 24 - 26 - 22 - 18 - 13 - 9 - 14 - 12 - 19 - 10 - 27 - 48 - 46 - 149 - 157 - 130 - 116 - 111 - 112 - 104 - 88 - 155 - 158 - 147 - 135 - 142 - 141 - 134 - 129 - 124 - 123 - 103 - 110 - 101 - 98 - 102 - 99 - 96 - 90 - 81 - 91 - 106 - 121 - 114 - 120 - 84 - 95 - 58 - 72 - 126 - 133 - 146 - 143 - 132 - 75 - 70 - 76 - 60 - 62 - 74 - 52 - 51 - 49 - 45 - 41 - 94 - 107 - 118 - 122 - 97 - 80 - 100 - 113 - 115 - 138 - 144 - 150 - 140 - 136 - 156 - 145 - 139 - 127 - 137 - 161 - 93 - 78 - 82 - 73 - 67 - 109 - 128 - 151 - 108 - 55 - 47 - 44 - 30 - 35 - 38 - 43 - 50 - 59 - 71 - 79 - 92 - 105 - 61 - 42 - 36 - 39 - 20 - 25 - 23 - 163 - 162 - 167 - 168 - 166 - 165 - 171 - 177 - 176 - 178 - 185 - 183 - 181 - 188 - 189 - 194 - 193 - 192 - 187 - 180 - 174 - 173 - 172 - 184 - 16 - 6 - 1 - 2 - 3 - 4 - 5 - 7 - 17 - 159 - 125 - 63 - 56 - 40 - 37 - 32 - 29 - 21 - 15 - 169 - 152 - 153 - 69 - 57

In [ ]:
_colonysiz = 10

f = open('/content/drive/My Drive/8° Período/Meta/lista/lista 2/resultados/Omã_Elitist_out_formiga.csv', 'w')

f.write(','.join(['Elitist_formiga_dist']))#, 'Elitist_formiga_dist', 'MaxMin_formiga_dist'
f.write('\n')

for i in range(0,_colonysiz):

  f.write(str(formigueiro1[i].distance) +"\n")# "," + str(formigueiro2[i].distance) + "," + str(formigueiro3[i].distance) + 
f.close()

In [ ]:
f = open('/content/drive/My Drive/8° Período/Meta/lista/lista 2/resultados/Omã_Elitist_out_tour.csv', 'w')

f.write(','.join(['Elitist_tour_dist']))#, 'Elitist_tour_dist', 'MaxMin_tour_dist'
f.write('\n')

for i in range(0,len(route1)):

  f.write(str(route1[i]) + "\n")#"," + str(route2[i]) + "," + str(route3[i]) + 
f.close()